In [1]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [12]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
# create a dataframe and import into pandas
df = pd.read_csv('../output_data/final_wine_data_172k.csv')

In [3]:
# check the top 5 of the dataframe
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
1,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
2,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
3,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
4,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem


In [4]:
# look at the string module
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

# Remove punctuation

In [5]:
#Function to remove Punctuation
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])# It will discard all punctuations
    return text_nopunct

df['descrip_clean'] = df['description'].apply(lambda x: remove_punct(x))

df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,descrip_clean
0,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,This is ripe and fruity a wine that is smooth ...
1,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,Tart and snappy the flavors of lime flesh and ...
2,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,Pineapple rind lemon pith and orange blossom s...
3,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,Much like the regular bottling from 2012 this ...
4,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,Blackberry and raspberry aromas show a typical...


# Tokenization

In [6]:
import re

# Function to Tokenize words
def tokenize(text):
    tokens = re.split('\W+', text) #W+ means that either a word character (A-Za-z0-9_) or a dash (-) can go there.
    return tokens

df['descript_tokenized'] = df['descrip_clean'].apply(lambda x: tokenize(x.lower())) 

#We convert to lower as Python is case-sensitive. 

df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,descrip_clean,descript_tokenized
0,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,This is ripe and fruity a wine that is smooth ...,"[this, is, ripe, and, fruity, a, wine, that, i..."
1,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,Tart and snappy the flavors of lime flesh and ...,"[tart, and, snappy, the, flavors, of, lime, fl..."
2,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,Pineapple rind lemon pith and orange blossom s...,"[pineapple, rind, lemon, pith, and, orange, bl..."
3,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,Much like the regular bottling from 2012 this ...,"[much, like, the, regular, bottling, from, 201..."
4,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,Blackberry and raspberry aromas show a typical...,"[blackberry, and, raspberry, aromas, show, a, ..."


In [7]:
# look at the column that is tokenized
print(df["descript_tokenized"])

0         [this, is, ripe, and, fruity, a, wine, that, i...
1         [tart, and, snappy, the, flavors, of, lime, fl...
2         [pineapple, rind, lemon, pith, and, orange, bl...
3         [much, like, the, regular, bottling, from, 201...
4         [blackberry, and, raspberry, aromas, show, a, ...
5         [heres, a, bright, informal, red, that, opens,...
6         [this, dry, and, restrained, wine, offers, spi...
7         [savory, dried, thyme, notes, accent, sunnier,...
8         [this, has, great, depth, of, flavor, with, it...
9         [soft, supple, plum, envelopes, an, oaky, stru...
10        [this, is, a, dry, wine, very, spicy, with, a,...
11        [slightly, reduced, this, wine, offers, a, cha...
12        [building, on, 150, years, and, six, generatio...
13        [zesty, orange, peels, and, apple, notes, abou...
14        [baked, plum, molasses, balsamic, vinegar, and...
15        [raw, blackcherry, aromas, are, direct, and, s...
16        [desiccated, blackberry, leath

# Remove stopwords

In [8]:
import nltk

stopword = nltk.corpus.stopwords.words('english')# All English Stopwords

In [9]:
# Function to remove Stopwords
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword]# To remove all stopwords
    return text

df['discript_nostop'] = df["descript_tokenized"].apply(lambda x: remove_stopwords(x))

df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,descrip_clean,descript_tokenized,discript_nostop
0,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,This is ripe and fruity a wine that is smooth ...,"[this, is, ripe, and, fruity, a, wine, that, i...","[ripe, fruity, wine, smooth, still, structured..."
1,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,Tart and snappy the flavors of lime flesh and ...,"[tart, and, snappy, the, flavors, of, lime, fl...","[tart, snappy, flavors, lime, flesh, rind, dom..."
2,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,Pineapple rind lemon pith and orange blossom s...,"[pineapple, rind, lemon, pith, and, orange, bl...","[pineapple, rind, lemon, pith, orange, blossom..."
3,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,Much like the regular bottling from 2012 this ...,"[much, like, the, regular, bottling, from, 201...","[much, like, regular, bottling, 2012, comes, a..."
4,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,Blackberry and raspberry aromas show a typical...,"[blackberry, and, raspberry, aromas, show, a, ...","[blackberry, raspberry, aromas, show, typical,..."


# Preprocessing Data: Using a Lemmatizer

In [13]:
wn = nltk.WordNetLemmatizer()

In [14]:
def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

df['descript_lemmatized'] = df['discript_nostop'].apply(lambda x: lemmatizing(x))

df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,descrip_clean,descript_tokenized,discript_nostop,descript_lemmatized
0,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,This is ripe and fruity a wine that is smooth ...,"[this, is, ripe, and, fruity, a, wine, that, i...","[ripe, fruity, wine, smooth, still, structured...","[ripe, fruity, wine, smooth, still, structured..."
1,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,Tart and snappy the flavors of lime flesh and ...,"[tart, and, snappy, the, flavors, of, lime, fl...","[tart, snappy, flavors, lime, flesh, rind, dom...","[tart, snappy, flavor, lime, flesh, rind, domi..."
2,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,Pineapple rind lemon pith and orange blossom s...,"[pineapple, rind, lemon, pith, and, orange, bl...","[pineapple, rind, lemon, pith, orange, blossom...","[pineapple, rind, lemon, pith, orange, blossom..."
3,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,Much like the regular bottling from 2012 this ...,"[much, like, the, regular, bottling, from, 201...","[much, like, regular, bottling, 2012, comes, a...","[much, like, regular, bottling, 2012, come, ac..."
4,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,Blackberry and raspberry aromas show a typical...,"[blackberry, and, raspberry, aromas, show, a, ...","[blackberry, raspberry, aromas, show, typical,...","[blackberry, raspberry, aroma, show, typical, ..."


In [15]:
help(nltk)

Help on package nltk:

NAME
    nltk

DESCRIPTION
    The Natural Language Toolkit (NLTK) is an open source Python library
    for Natural Language Processing.  A free online book is available.
    (If you use the library for academic research, please cite the book.)
    
    Steven Bird, Ewan Klein, and Edward Loper (2009).
    Natural Language Processing with Python.  O'Reilly Media Inc.
    http://nltk.org/book
    
    @version: 3.4.3

PACKAGE CONTENTS
    app (package)
    book
    ccg (package)
    chat (package)
    chunk (package)
    classify (package)
    cluster (package)
    collections
    collocations
    compat
    corpus (package)
    data
    decorators
    downloader
    draw (package)
    featstruct
    grammar
    help
    inference (package)
    internals
    jsontags
    lazyimport
    lm (package)
    metrics (package)
    misc (package)
    parse (package)
    probability
    sem (package)
    sentiment (package)
    stem (package)
    tag (package)
    tbl (pac

# Saving the cleaned Data

In [16]:
# We save the file as CSV 
df.to_csv("../output_data/final_wine_data_172k_clean.csv", index = False)